# Can we use Claude to extract all the fields from a form

Based on the idea from here: 
https://github.com/co-cddo/form-extractor-prototype_testing/tree/main

Yes we can.

In [1]:

%load_ext autoreload
%autoreload 2

import logging
import json

import boto3
import nest_asyncio
from anthropic import AsyncAnthropicBedrock
from dotenv import load_dotenv

from form_processor import run_form_processing, test_client, load_batch_results, compute_total

logging.basicConfig(
    filename="form_processing.log",
    level=logging.INFO,  # Adjust the level as needed (e.g., INFO, DEBUG, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a",  # 'a' for append mode, 'w' for overwrite mode
)
logging.info("Setup log")

ImportError: cannot import name 'test_client' from 'form_processor' (/Users/david.gillespie/Code/c-af-form-experiment/form_processor.py)

In [ ]:

logging.info("Intitiating log")

load_dotenv()

nest_asyncio.apply()

boto3.setup_default_session()
client = AsyncAnthropicBedrock()
test_client(client)


In [ ]:

with open("extract-form-questions.json", "r") as file:
    json_string = file.read()

extraction_tool = json.loads(json_string)

folder_path = "forms_scrape"
results = run_form_processing(
    folder_path=folder_path,
    client=client,
    extraction_tool=extraction_tool,
    batch_size=10,
    max_concurrent=10,
    results_file="test_folder.pickle",
)



In [ ]:
results = load_batch_results("test_folder.pickle")

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
compute_total(results_df)
